# Private Comparison Protocols

Author: 
- Théo Ryffel - [Twitter](http://twitter.com/theoryffel) - [GitHub](https://github.com/LaRiffle)

Credits:
- David Pointcheval - [Link to lectures](https://www.di.ens.fr/david.pointcheval/index.php#cslides)

Comparison is today the key building block to design neural networks: every ReLU activation function can be written as such: $x \to x \cdot (x > 0)$. But comparisons are notoriously hard over encrypted data! We will explore in the context of the Yao's Millionaires' problem some mechanisms to perform private equality and private comparison, that will provide insights on how ReLU activations can be implemented.

Let's start with equality, that will give us the basic tools to perform private comparison.

### Equality

Let's assume we only have two parties. Alice owns a value $x$ and Bob owns a value $y$, that we assume to live in $\mathbb{Z}_{2^{n}}$. Those are either integers or float already converted to fixed precision. Alice and Bob want to learn whether $x = y$ or not.

$\newcommand{\shared}[1]{[\![ #1 ]\!]}$
To do so, they can follow these steps:
- Alice and Bob secret share their values to build $\shared{x}$ and $\shared{y}$
- Each party also sample separately a non-zero random value $r_i$ and secret share it to obtain $\shared{r_i}$, with $i=1,2$
- Then, they jointly compute $\shared{z} = (\shared{x} - \shared{y}) \cdot \shared{r_1} \cdot \shared{r_2}$
- Last, they reconstruct $z$: it is 0 if and only if $x = y$, else it is a random value 

As you see, the idea is quite simple! They compute the difference, and each party mask it with a non-zero random value, so that in case $x \neq y$, the value reconstructed is completely random.

Let's implement it, we have everything we need!

 We will assume that we can use the following functions:
- `random_mask() -> int`, that computes a non-zero random value that we can use as a mask
- `secret_share(x: int) -> List[int]`, that builds shares $\shared{x}$ for some $x$
- `decrypt(shares: List[int]) -> int`, that reconstructs $x$ from its shares $\shared{x}$
- `sub(x_shares: List[int], y_shares: List[int]) -> List[int]`, that given $\shared{x}$ and $\shared{y}$ returns $\shared{x - y}$
- `mul(x_shares: List[int], y_shares: List[int]) -> List[int]`, that given $\shared{x}$ and $\shared{y}$ returns $\shared{x * y}$

We will also assume that the shares live in $\mathbb{Z}_{2^{32}}$.

In [1]:
from quiz import random_mask, secret_share, decrypt, sub, mul

In [4]:
def equality(x: int, y: int) -> int:
    # Add here the code for the the 4 steps describing how to perform private equality
    x_sh = secret_share(x)
    y_sh = secret_share(y)
    
    r1 = random_mask()
    r1_sh = secret_share(r1)
    r2 = random_mask()
    r2_sh = secret_share(r2)
    
    aux1_sh = sub(x_sh, y_sh)
    aux2_sh = mul(aux1_sh, r1_sh)
    z_sh = mul(aux2_sh, r2_sh)
    
    z = decrypt(z_sh)
    return z 

In [5]:
# we import the answer checker
from quiz import test_equality

test_equality.check(equality)

Test failed: equality(3, 3) should output 0, not 1


Let's see how it performs:

In [6]:
print(equality(3, 3))
print(equality(2, 4))
print(equality(22, -14))

1
298276705
-114061451


Assuming that the inputs $x$ and $y$ are already shared, alonside with the $r_i$, how many communication rounds are needed to compute $\shared{z}$?

In [4]:
from quiz import test_rounds

# Fill the gap!
test_rounds.check(__)

Test failed: Expected to get 2, you passed False instead.


False

### Comparison

Ok, now that we have discovered the key ideas in the equality computation, we will dive more in depth in comparison!

Remember that Alice and Bob respectively own $x$ and $y$. They now want to learn whether $x \leq y$.

_We denote $x_{n-1}...x_0$ the binary decomposition of x in  $\mathbb{Z}_{2^{n}}$_

To do so, we will first use the following theorem:

> **Theorem**
>
> Given $x = x_{n-1}...x_0$ and $y = y_{n-1}...y_0$, <br>
> if we define the following ensembles: <br>
> $$ T_x^1 = \{x_{n-1}...x_i|x_i=1\} \qquad T_y^0 = \{y_{n-1}...y_{i+1}1|y_i=0\} $$
> 
> namely the ensembles composed respectively of
> - chains of the first bits of the decomposition of $x$, such that the last bit of the chain is 1
> - chains of the first bits of the decomposition of $y$, such that the last bit of the chain is 0, but is replaced with 1
>
> Then:
> $$ x > y \Leftrightarrow T_x^1 \cap T_y^0 \neq \emptyset $$
>
> namely $x > y$ if and only if $x$ and $y$ share a common sub-decomposition in $T_x^1$ and $T_y^0$

Informally, the idea here is that you have two numbers decomposed as $x = a_{n-1}...a_{l+1}1x_{l-1}...x_0$ and $y = a_{n-1}...a_{l+1}0y_{l-1}...y_0$ (for some $l$), then we will have $x > y$, and $a_{n-1}...a_{l+1}1$ will both be in $T_x^1$ and $T_y^0$.

Proof:
$$ x > y \Leftrightarrow \exists ! i < n,  (x_i > y_i) \wedge (\forall j > i, x_j = y_j)$$
$$ \Leftrightarrow \exists ! i < n,  (x_i = 1) \wedge (y_i = 0) \wedge (\forall j > i, x_j = y_j)$$
$$ \Leftrightarrow \exists ! i < n,  (y_i = 0) \wedge (x_{n-1}...x_i = y_{n-1}...y_{i+1}1)$$
$$ \Leftrightarrow \left | T_x^1 \cap T_y^0 \right | = 1$$

We need a last transformation: we fill and order the sets $T_x^1 = \{ X_i \}$ and $T_y^0 = \{ Y_i \}$, where:
- if $x_i = 0$, $X_i = 2^n$, otherwise $X_i = x_{n-1}...x_i \in [0, 2^{n-i}[$
- if $y_i = 1$, $Y_i = 2^n + 1$, otherwise $Y_i = y_{n-1}...y_{i+1}1 \in [0, 2^{n-i}[$

Hence, we have 
$$ x > y \Leftrightarrow \exists ! i < n, X_i = Y_i $$

It is now very easy to reuse what we have done for equality to check whether $X_i = Y_i$ for all $i$!

We will therefore perform all these $n$ checks in parallel, and if one decrypts to 0 we will know that $x > y$. If all decrypt to random, then we know that $x \leq y$.

Assuming that the inputs $X_i$ and $Y_i$ are already shared, alonside with all the $r$ needed, how many communication rounds are needed to compute $x > y$?

In [1]:
from quiz import test_comp_rounds

# Fill the gap!
test_comp_rounds.check(__)

Test failed: Expected to get 2, you passed  instead.


False

This was quite dense! But congrats, you went through all this and know have understood how we can implement private comparison to solve the Yao's Millionaire' problem!

### Extension to private machine learning

But how does this relates to machine learning? Unfortunately, you can't use this protocol directly to implement the ReLU activation. Indeed, in the Yao Millionaire problem, all users know their input in cleartext, while in a neural network, one should be able to apply the comparison from the ReLU on the result secret shared of the previous layer.

In addition, the result is also given in cleartext: if we take the case of the equality test, you need to reconstruct the output and check whether it is equal to 0 or random to derive the result (true when it is 0, false otherwise). It means that you can’t take the secret share value « as is » and use it as an input of another computation.

Fortunately, there are many works that have been developed that implement secure comparison on secret shared inputs that output secret shared results and can be plugged in neural networks. Describing them in detail is beyond the scope of this lesson, but we can give a non-exhaustive list of techniques that we have studied here at OpenMined:

- **ABY**, a 2-party semi-honest protocol that mixes Arithmetic sharing, Binary sharing and Yao’s garbled circuits (2015) [See full paper](https://encrypto.de/papers/DSZ15.pdf)
- **SecureNN**, a 3-party semi-honest protocol that proposes advanced implementations of neural network blocks (2018) [See full paper](https://eprint.iacr.org/2018/442.pdf)
- **FALCON**, a 3-party honest-majority maliciously secure protocol which provide benchmarks on neural nets like VGG16 (2020) [See full paper](https://arxiv.org/pdf/2004.02229.pdf)
- **AriaNN**, a 2-party semi-honest that implements comparison with arithmetic sharing with a single communication round (2020) [See full paper](https://arxiv.org/pdf/2006.04593.pdf)

We have implemented a couple of this algorithms in [SyMPC](https://github.com/OpenMined/SyMPC), our library for efficient MPC that works together with [PySyft](https://github.com/OpenMined/PySyft), and we will see in the next section how to use it to perform encrypted machine learning seamlessly!
